In [50]:
import pickle
import os

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.signal import resample

from datetime import datetime

In [51]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [52]:
full_df = pd.read_csv("/content/drive/MyDrive/NDS_2023/Capstone-project-0223-healthfood/Capstone-Retail/data_dsmarket/aux/item_sales_calendar_prices.csv")
full_df.head()

<ipython-input-52-407c5a75f90d>:1: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  full_df = pd.read_csv("/content/drive/MyDrive/NDS_2023/Capstone-project-0223-healthfood/Capstone-Retail/data_dsmarket/aux/item_sales_calendar_prices.csv")


,Unnamed: 0,id,item,category,department,store,store_code,region,date,units,weekday,weekday_int,event,day,week,month,month_int,year,year_week,sell_price
0,0,ACCESORIES_1_008_NYC_1,ACCESORIES_1_008,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,2011-01-29,12.0,Saturday,1,NaN,29,4,January,1,2011,2011-4,NaN
1,1,ACCESORIES_1_009_NYC_1,ACCESORIES_1_009,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,2011-01-29,2.0,Saturday,1,NaN,29,4,January,1,2011,2011-4,NaN
2,2,ACCESORIES_1_015_NYC_1,ACCESORIES_1_015,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,2011-01-29,4.0,Saturday,1,NaN,29,4,January,1,2011,2011-4,NaN
3,3,ACCESORIES_1_016_NYC_1,ACCESORIES_1_016,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,2011-01-29,5.0,Saturday,1,NaN,29,4,January,1,2011,2011-4,NaN
4,4,ACCESORIES_1_022_NYC_1,ACCESORIES_1_022,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,2011-01-29,2.0,Saturday,1,NaN,29,4,January,1,2011,2011-4,NaN


In [53]:
full_df.drop(["weekday_int", "Unnamed: 0"], axis=1, inplace=True)

In [54]:
full_df.shape

(18550276, 18)

### Date

In [55]:
full_df_gb_date = full_df.groupby(["id", "item", "category", "department", "store", "region", "date","year_week", "month", "year"]).agg({
    "units": np.sum,  # Compute sum of "units" column
    "sell_price": np.mean  # Compute mean of "sell_price" column
})
full_df_gb_date.reset_index(inplace=True)

In [56]:
full_df_gb_date.shape

(18550276, 12)

In [57]:
full_df_gb_date['sell_price'].isnull().sum()

3520473

In [58]:
full_df_gb_date[full_df_gb_date['sell_price'].isnull()]

,id,item,category,department,store,region,date,year_week,month,year,units,sell_price
30,ACCESORIES_1_001_BOS_1,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,South_End,Boston,2014-01-01,2014-1,January,2014,1.0,NaN
31,ACCESORIES_1_001_BOS_1,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,South_End,Boston,2014-01-19,2014-3,January,2014,1.0,NaN
32,ACCESORIES_1_001_BOS_1,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,South_End,Boston,2014-01-21,2014-4,January,2014,2.0,NaN
33,ACCESORIES_1_001_BOS_1,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,South_End,Boston,2014-01-23,2014-4,January,2014,1.0,NaN
34,ACCESORIES_1_001_BOS_1,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,South_End,Boston,2014-02-05,2014-6,February,2014,1.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
18550246,SUPERMARKET_3_827_PHI_3,SUPERMARKET_3_827,SUPERMARKET,SUPERMARKET_3,Queen_Village,Philadelphia,2016-02-29,2016-9,February,2016,2.0,NaN
18550247,SUPERMARKET_3_827_PHI_3,SUPERMARKET_3_827,SUPERMARKET,SUPERMARKET_3,Queen_Village,Philadelphia,2016-03-02,2016-9,March,2016,5.0,NaN
18550248,SUPERMARKET_3_827_PHI_3,SUPERMARKET_3_827,SUPERMARKET,SUPERMARKET_3,Queen_Village,Philadelphia,2016-03-03,2016-9,March,2016,4.0,NaN
18550249,SUPERMARKET_3_827_PHI_3,SUPERMARKET_3_827,SUPERMARKET,SUPERMARKET_3,Queen_Village,Philadelphia,2016-03-04,2016-9,March,2016,1.0,NaN


In [59]:
full_df_gb_date["sell_price"] = full_df_gb_date["sell_price"].round(2)

In [60]:
full_df_gb_date["sell_price"] = full_df_gb_date.groupby("id", group_keys=False)["sell_price"].apply(lambda x: x.ffill())
full_df_gb_date["sell_price"] = full_df_gb_date.groupby("id", group_keys=False)["sell_price"].apply(lambda x: x.bfill())
full_df_gb_date.head(20)

,id,item,category,department,store,region,date,year_week,month,year,units,sell_price
0,ACCESORIES_1_001_BOS_1,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,South_End,Boston,2013-07-16,2013-29,July,2013,1.0,10.99
1,ACCESORIES_1_001_BOS_1,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,South_End,Boston,2013-07-17,2013-29,July,2013,3.0,10.99
2,ACCESORIES_1_001_BOS_1,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,South_End,Boston,2013-07-26,2013-30,July,2013,3.0,10.99
3,ACCESORIES_1_001_BOS_1,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,South_End,Boston,2013-07-27,2013-30,July,2013,1.0,10.99
4,ACCESORIES_1_001_BOS_1,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,South_End,Boston,2013-07-29,2013-31,July,2013,2.0,10.99
5,ACCESORIES_1_001_BOS_1,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,South_End,Boston,2013-08-09,2013-32,August,2013,1.0,10.99
6,ACCESORIES_1_001_BOS_1,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,South_End,Boston,2013-08-27,2013-35,August,2013,1.0,10.99
7,ACCESORIES_1_001_BOS_1,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,South_End,Boston,2013-08-29,2013-35,August,2013,1.0,10.99
8,ACCESORIES_1_001_BOS_1,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,South_End,Boston,2013-09-02,2013-36,September,2013,2.0,10.99
9,ACCESORIES_1_001_BOS_1,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,South_End,Boston,2013-09-07,2013-36,September,2013,1.0,10.99


In [61]:
full_df_gb_date["revenue"] = full_df_gb_date["units"]*full_df_gb_date["sell_price"]
full_df_gb_date.head(20)

,id,item,category,department,store,region,date,year_week,month,year,units,sell_price,revenue
0,ACCESORIES_1_001_BOS_1,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,South_End,Boston,2013-07-16,2013-29,July,2013,1.0,10.99,10.99
1,ACCESORIES_1_001_BOS_1,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,South_End,Boston,2013-07-17,2013-29,July,2013,3.0,10.99,32.97
2,ACCESORIES_1_001_BOS_1,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,South_End,Boston,2013-07-26,2013-30,July,2013,3.0,10.99,32.97
3,ACCESORIES_1_001_BOS_1,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,South_End,Boston,2013-07-27,2013-30,July,2013,1.0,10.99,10.99
4,ACCESORIES_1_001_BOS_1,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,South_End,Boston,2013-07-29,2013-31,July,2013,2.0,10.99,21.98
5,ACCESORIES_1_001_BOS_1,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,South_End,Boston,2013-08-09,2013-32,August,2013,1.0,10.99,10.99
6,ACCESORIES_1_001_BOS_1,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,South_End,Boston,2013-08-27,2013-35,August,2013,1.0,10.99,10.99
7,ACCESORIES_1_001_BOS_1,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,South_End,Boston,2013-08-29,2013-35,August,2013,1.0,10.99,10.99
8,ACCESORIES_1_001_BOS_1,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,South_End,Boston,2013-09-02,2013-36,September,2013,2.0,10.99,21.98
9,ACCESORIES_1_001_BOS_1,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,South_End,Boston,2013-09-07,2013-36,September,2013,1.0,10.99,10.99


In [62]:
full_df_gb_date.to_csv("/content/drive/MyDrive/NDS_2023/Capstone-project-0223-healthfood/Capstone-Retail/data_dsmarket/aux/full_df_gb_date.csv")

### Time series

In [64]:
full_df_gb_date_2 = full_df_gb_date.groupby(["item", "category", "department", "store", "region", "date"]).agg({
    "units": np.sum,  # Compute sum of "units" column
    "sell_price": np.mean,  # Compute mean of "sell_price" column
    "revenue": np.sum,  # Compute sum of "revenue" column
})
full_df_gb_date_2.reset_index(inplace=True)

### Year_week

In [21]:
full_df_gb_yearweek = full_df.groupby(["id", "item", "category", "department", "store", "region", "year_week", "month", "year"]).agg({
    "units": np.sum,  # Compute sum of "units" column
    "sell_price": np.mean  # Compute mean of "sell_price" column
})

In [7]:
full_df_gb_yearweek.shape

(5606994, 2)

In [22]:
full_df_gb_yearweek.reset_index(inplace=True)
full_df_gb_yearweek.head(20)

,id,item,category,department,store,region,year_week,month,year,units,sell_price
0,ACCESORIES_1_001_BOS_1,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,South_End,Boston,2013-29,July,2013,4.0,10.9858
1,ACCESORIES_1_001_BOS_1,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,South_End,Boston,2013-30,July,2013,4.0,10.9858
2,ACCESORIES_1_001_BOS_1,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,South_End,Boston,2013-31,July,2013,2.0,10.9858
3,ACCESORIES_1_001_BOS_1,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,South_End,Boston,2013-32,August,2013,1.0,10.9858
4,ACCESORIES_1_001_BOS_1,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,South_End,Boston,2013-35,August,2013,2.0,10.9858
5,ACCESORIES_1_001_BOS_1,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,South_End,Boston,2013-36,September,2013,3.0,10.9858
6,ACCESORIES_1_001_BOS_1,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,South_End,Boston,2013-39,September,2013,1.0,10.9858
7,ACCESORIES_1_001_BOS_1,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,South_End,Boston,2013-41,October,2013,2.0,10.9858
8,ACCESORIES_1_001_BOS_1,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,South_End,Boston,2013-42,October,2013,1.0,10.9858
9,ACCESORIES_1_001_BOS_1,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,South_End,Boston,2013-43,October,2013,1.0,10.9858


In [14]:
full_df_gb_yearweek["sell_price"] = full_df_gb_yearweek["sell_price"].round(2)

In [19]:
full_df_gb_yearweek["sell_price"] = full_df_gb_yearweek.groupby("id", group_keys=False)["sell_price"].apply(lambda x: x.ffill())
full_df_gb_yearweek["sell_price"] = full_df_gb_yearweek.groupby("id", group_keys=False)["sell_price"].apply(lambda x: x.bfill())
full_df_gb_yearweek.head(20)

<ipython-input-19-0af3ad79035e>:1: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  full_df_gb_yearweek["sell_price"] = full_df_gb_yearweek.groupby("id")["sell_price"].apply(lambda x: x.ffill())
<ipython-input-19-0af3ad79035e>:2: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  full_df_gb_yearweek["se

In [9]:
full_df_gb_yearweek["revenue"] = full_df_gb_yearweek["units"]*full_df_gb_yearweek["sell_price"]
full_df_gb_yearweek.head(20)

,id,item,category,department,store,region,year_week,month,year,units,sell_price,revenue
0,ACCESORIES_1_001_BOS_1,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,South_End,Boston,2013-29,July,2013,4.0,10.9858,43.9432
1,ACCESORIES_1_001_BOS_1,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,South_End,Boston,2013-30,July,2013,4.0,10.9858,43.9432
2,ACCESORIES_1_001_BOS_1,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,South_End,Boston,2013-31,July,2013,2.0,10.9858,21.9716
3,ACCESORIES_1_001_BOS_1,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,South_End,Boston,2013-32,August,2013,1.0,10.9858,10.9858
4,ACCESORIES_1_001_BOS_1,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,South_End,Boston,2013-35,August,2013,2.0,10.9858,21.9716


In [15]:
full_df_gb_yearweek.shape

(5606994, 12)

In [11]:
full_df_gb_yearweek.to_csv("/content/drive/MyDrive/NDS_2023/Capstone-project-0223-healthfood/Capstone-Retail/data_dsmarket/powerbi/full_df_gb_yearweek.csv",header=True)

In [ ]:
full_df_gb_month = item_sales_calendar_prices.groupby(["id", "item", "category", "department", "store", "region", "weekday", "event", "month", "year"]).sum(numeric_only=True)
full_df_gb_month.reset_index(inplace=True)
full_df_gb_month.head(10)

In [ ]:
sales_calendar_prices.head()

,year_week,id,units,item,category,department,store,store_code,region,date,sell_price,year,week
0,2011-10,ACCESORIES_1_001_BOS_1,0,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,South_End,BOS_1,Boston,2011-03-07,NaN,NaN,NaN
1,2011-10,ACCESORIES_1_001_BOS_2,0,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,Roxbury,BOS_2,Boston,2011-03-07,NaN,NaN,NaN
2,2011-10,ACCESORIES_1_001_BOS_3,0,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,Back_Bay,BOS_3,Boston,2011-03-07,NaN,NaN,NaN
3,2011-10,ACCESORIES_1_001_NYC_1,0,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,2011-03-07,NaN,NaN,NaN
4,2011-10,ACCESORIES_1_001_NYC_2,0,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,Harlem,NYC_2,New York,2011-03-07,NaN,NaN,NaN


In [ ]:
sales_calendar_prices.to_csv("/content/drive/MyDrive/NDS_2023/Capstone-project-0223-healthfood/Capstone-Retail/data_dsmarket/aux/sales_calendar_prices.csv")